# Exploring Movie Industry Insights for Microsoft Movie Studio

## Overview

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. We are charged with exploring what types of films are currently doing the best at the box office. We must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

## Business Problem

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

## Data Understanding

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# We will be working primairly with three datasets.
# The first are easy to import since they are stored as files with comma-seperated values
box_office = pd.read_csv('data/bom.movie_gross.csv')
movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

In [3]:
# The other data source is a SQLite database from the Internet Movie Database (IMDB).
conn = sqlite3.connect('data/im.db')

In [4]:
box_office.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### The IMDB Data
The IMDB data is composed of 8 different tables. Below is its ERD.

![IMDB ERD](images/imdb_erd.jpeg)

This dataset is the most roboust and likely contains more information than we need. A quick look at the movie_basics table shows us that the dataset contains information on 146,144 unique records between the years of 2010 and 2115 (which is either an error or at the very least something to look into when we do data cleaning).

Looking at the other tables in the databse, it looks like we also have information on each movie's: staff (actors, producers, writers, etc.), names of chartacters played by actors, some basic information on the people involved (like birth year and year of death), as well as info on movie ratings.

In [6]:
q = """
SELECT MIN(start_year), MAX(start_year), COUNT(DISTINCT movie_id)
FROM movie_basics
;"""

pd.read_sql(q, conn)

,MIN(start_year),MAX(start_year),COUNT(DISTINCT movie_id)
0,2010,2115,146144


In [7]:
q = """
SELECT *
FROM principals
GROUP BY category
LIMIT 10
;"""

pd.read_sql(q, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
2,tt1179035,1,nm3743659,archive_footage,None,"[""Himself""]"
3,tt1754621,4,nm5080976,archive_sound,None,"[""Himself (videographer)""]"
4,tt0323808,9,nm0676104,cinematographer,None,None
5,tt0323808,8,nm0779346,composer,None,None
6,tt0111414,2,nm0398271,director,None,None
7,tt0323808,10,nm0059247,editor,None,None
8,tt0111414,3,nm3739909,producer,producer,None
9,tt10229602,10,nm3193762,production_designer,None,None


### Box Office Data

The box office comes from Box Office Mojo. It contains records on 3,386 unique movie titles between the years of 2010 and 2018. For each movie, it provides us with the movie's title, the studio which produced it, the domestic box office gross earnings, foreign box office gross earnings and the year of release.

In [8]:
box_office.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [9]:
# Get the range of years in the box office dataset

print('The data sets begins in the year:', box_office['year'].min())
print('The dataset contains data on movies through the year: ', box_office['year'].max())

box_office.groupby('studio').count()

The data sets begins in the year: 2010
The dataset contains data on movies through the year:  2018


,title,domestic_gross,foreign_gross,year
studio,,,,
3D,1,1,1,1
A23,2,2,0,2
A24,49,49,19,49
ADC,2,2,0,2
AF,6,6,2,6
...,...,...,...,...
XL,2,2,0,2
YFG,1,1,0,1
Yash,14,13,7,14


In [10]:
# How many unique movies titles are in the dataset?
box_office['title'].nunique()

3386

### Movie Budgets Data

The movie budgets data comes from The-Numbers.com. It contains records on 5,698 unique movie titles between the years of 1915 and 2020. For each movie, it provides us with the movie's release date, its title, its production budget, and its foreign and domestic box office earnings.

In [11]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [12]:
# How many unique movies titles are in the dataset?
movie_budgets['movie'].nunique()

5698

In [13]:
# Get the range of years in the box office dataset

movie_budgets['year'] = movie_budgets['release_date'].str[-4:]

print('The data sets begins in the year:', movie_budgets['year'].min())
print('The dataset contains data on movies through the year: ', movie_budgets['year'].max())

The data sets begins in the year: 1915
The dataset contains data on movies through the year:  2020


## Data Preperation

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

### Data Cleaning
Lorem ipsum dolor sit amet, consectetur adipiscing elit.

Looking at the box_office dataset first, it looks like there are some missing values. The movie studio is missing on X records, domestic_gross is missing on 28 records and foreign_gross is missing on 1,350 records.

In [14]:
box_office.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

The missing studio and domestic_gross records are small. If we had to drop them it wouldn't make much of a difference.
However, we are missing foreign_gross on a substantial chunk of the dataset so it's worth taking a closer look at those values.

In [15]:
box_office[box_office['foreign_gross'].isna()]

,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


Taking 'Flipped' as an example, it looks like this data is really missing. Looking at The Numbers website for this movie it DID get a international release. Furthermore, if we look at the same movie in our movie_budgets dataset, we can see complete data there. Moreover, the data on domestic and foreign gross here looks more precise and appears to be rounded in the box_office dataset. Because of that, we can just rely on data on worldwide gorss sales using the movie_budgets source. We will use box_office just to get the studio names for each movie.

In [16]:
movie_budgets[movie_budgets['movie'].str.contains("Flipped")]

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
3221,22,"Aug 6, 2010",Flipped,"$14,000,000","$1,755,212","$4,289,993",2010


Next, the box_office title column also contains some entries that aren't the film's title. For example, Alice in Wonderland (2010) has '(2010)' in the title despite it not being part of the original title. We want to find those instances and clean them up.

In [24]:
replacement = ['\(2010\)','\(2011\)','\(2012\)','\(2013\)','\(2014\)','\(2015\)','\(2016\)','\(2017\)','\(2018\)',]
box_office['title'] = box_office['title'].replace(replacement,"", regex=True)
box_office[['title_cleaned','title_notes']] = box_office.title.str.split(pat='\(',expand=True)

# I will also run a strip just in case we were left with any unwanted whitespace after the last operation

box_office['title_cleaned'] = box_office['title_cleaned'].str.strip()

Let's take a closer look at movie_budgets next. Initially, it looks like the dataset has no misisng values. However, just because it doesn't have Null values doesn't mean some values don't have a placeholder. So we'll investigate this dataset more closely.

In [18]:
movie_budgets['worldwide_gross'].value_counts().sort_values(ascending=False)

$0              367
$8,000,000        9
$2,000,000        6
$7,000,000        6
$4,000,000        4
               ... 
$29,200,000       1
$17,127           1
$26,097,497       1
$234,723,148      1
$8,378,141        1
Name: worldwide_gross, Length: 5356, dtype: int64

worldwide_gross has 367 occourances of $0. That's the only unusual high count occourance. Let's take a look at those records.

Before we do that, it will be helpful to convert the values in these numeric columns from objects to floats.

In [19]:
movie_budgets[movie_budgets.columns[3:]] = movie_budgets[movie_budgets.columns[3:]].replace('[\$,]', '', regex=True).astype(float)

# Now let's look at those records where the worldwide_gross is equal to 0

movie_budgets_zero = movie_budgets.loc[movie_budgets['worldwide_gross'] == 0]

movie_budgets_zero

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
194,95,"Dec 31, 2020",Moonfall,150000000.0,0.0,0.0,2020.0
479,80,"Dec 13, 2017",Bright,90000000.0,0.0,0.0,2017.0
480,81,"Dec 31, 2019",Army of the Dead,90000000.0,0.0,0.0,2019.0
535,36,"Feb 21, 2020",Call of the Wild,82000000.0,0.0,0.0,2020.0
670,71,"Aug 30, 2019",PLAYMOBIL,75000000.0,0.0,0.0,2019.0
...,...,...,...,...,...,...,...
5761,62,"Dec 31, 2014",Stories of Our Lives,15000.0,0.0,0.0,2014.0
5764,65,"Dec 31, 2007",Tin Can Man,12000.0,0.0,0.0,2007.0
5771,72,"May 19, 2015",Family Motocross,10000.0,0.0,0.0,2015.0
5777,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.0,2018.0


Some of the above results are streaming releases, meaning they never had a box office. There, a value of 0 makes sense. This applies, for example, to Bright. However, for other movies the data is just missing. For example, a quick search shows that Army of the Dead (released Dec 31, 2019) made $1 million at the Box Offce.

Since this is just 367 rows out of 5,782 I'm deciding to simply drop these values. Lastly, I also rename the column "movie" to "title" so that it more closely matches the other dataframes we're working with.

In [20]:
# movie_budgets_clean = movie_budgets[movie_budgets['worldwide_gross'] > 0]
movie_budgets.drop(movie_budgets[movie_budgets.worldwide_gross == 0].index, inplace=True)
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5415 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5415 non-null   int64  
 1   release_date       5415 non-null   object 
 2   movie              5415 non-null   object 
 3   production_budget  5415 non-null   float64
 4   domestic_gross     5415 non-null   float64
 5   worldwide_gross    5415 non-null   float64
 6   year               5415 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 338.4+ KB


### Merging Data Sets
The three data sources we want to merge do not have a common key. Therefore we need to create one.

While movie titles may repeat, it's rarer for a movie title to repeat within the same year. So, we will create a dataframe using a Title/Year pairing.

Because movie_budgets has the most relevant information we're after about a movie's performence (its budget and revenue) we will join the other data sources onto it.

In [26]:
movie_budgets['year'] = movie_budgets['release_date'].str[-4:]
movie_budgets['title/year'] = movie_budgets['movie'] + '/' + movie_budgets['year']
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,title/year
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2009,Avatar/2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,Pirates of the Caribbean: On Stranger Tides/2011
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,Dark Phoenix/2019
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,Avengers: Age of Ultron/2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,Star Wars Ep. VIII: The Last Jedi/2017


In [25]:
box_office['title/year'] = box_office['title_cleaned'] + '/' + box_office['year'].astype(str)
box_office.head()

,title,studio,domestic_gross,foreign_gross,year,title_cleaned,title_notes,title/year
0,Toy Story 3,BV,415000000.0,652000000,2010,Toy Story 3,None,Toy Story 3/2010
1,Alice in Wonderland,BV,334200000.0,691300000,2010,Alice in Wonderland,None,Alice in Wonderland/2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010,Harry Potter and the Deathly Hallows Part 1,None,Harry Potter and the Deathly Hallows Part 1/2010
3,Inception,WB,292600000.0,535700000,2010,Inception,None,Inception/2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010,Shrek Forever After,None,Shrek Forever After/2010


In [30]:
first_merge = pd.merge(movie_budgets,box_office, how="left", on='title/year')
first_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5416 entries, 0 to 5415
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5416 non-null   int64  
 1   release_date       5416 non-null   object 
 2   movie              5416 non-null   object 
 3   production_budget  5416 non-null   float64
 4   domestic_gross_x   5416 non-null   float64
 5   worldwide_gross    5416 non-null   float64
 6   year_x             5416 non-null   object 
 7   title/year         5416 non-null   object 
 8   title              1327 non-null   object 
 9   studio             1326 non-null   object 
 10  domestic_gross_y   1325 non-null   float64
 11  foreign_gross      1158 non-null   object 
 12  year_y             1327 non-null   float64
 13  title_cleaned      1327 non-null   object 
 14  title_notes        3 non-null      object 
dtypes: float64(5), int64(1), object(9)
memory usage: 677.0+ KB


Lastly, we want to merge in data from IMBD into this dataframe.

In [67]:
# q = """
# SELECT mb.primary_title AS title, mb.genres, mb.start_year AS year, p.primary_name AS director
# FROM movie_basics AS mb
# JOIN directors AS d
# USING(movie_id)
# JOIN persons AS p
# USING(person_id)
# WHERE director LIKE "%Sam Raimi%"
# ;"""

# imdb_data = pd.read_sql(q, conn)
# imdb_data


q = """
SELECT title
FROM movie_akas
WHERE title LIKE "%Spider-Man%"
;"""

imdb_data = pd.read_sql(q, conn)
imdb_data

,title
0,The Amazing Spider-Man 2
1,The Amazing Spider-Man 2
2,The Amazing Spider-Man 2
3,Neverjetni Spider-Man 2
4,The Amazing Spider-Man 2: El poder de Electro
...,...
97,Spider-Man: Into the Spider-Verse
98,Spider-Man: Kohti Hämähäkkiversumia
99,Spider-Man: New Generation
100,Spider-Man: Novi svijet


In [52]:
imdb_data['title/year'] = imdb_data['title'] + '/' + imdb_data['year'].astype(str)

In [53]:
combined_data = pd.merge(first_merge,imdb_data, how="left", on='title/year')

In [55]:
combined_data[combined_data['director'].isna()]

,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,year_x,title/year,title_x,studio,domestic_gross_y,foreign_gross,year_y,title_cleaned,title_notes,title_y,genres,year,director
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2009,Avatar/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,Star Wars Ep. VIII: The Last Jedi/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2015,Star Wars Ep. VII: The Force Awakens/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000.0,309420425.0,9.634204e+08,2007,Pirates of the Caribbean: At Worldâs End/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,16,"May 4, 2007",Spider-Man 3,258000000.0,336530303.0,8.948602e+08,2007,Spider-Man 3/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5411,76,"May 26, 2006",Cavite,7000.0,70071.0,7.164400e+04,2006,Cavite/2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5412,77,"Dec 31, 2004",The Mongol King,7000.0,900.0,9.000000e+02,2004,The Mongol King/2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5413,79,"Apr 2, 1999",Following,6000.0,48482.0,2.404950e+05,1999,Following/1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5414,80,"Jul 13, 2005",Return to the Land of Wonders,5000.0,1338.0,1.338000e+03,2005,Return to the Land of Wonders/2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Analysis

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

## Conclusions

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas in vulputate orci, eget posuere massa. Cras at lectus vel diam porta viverra. Ut vehicula congue aliquam. Suspendisse finibus risus quis hendrerit rhoncus. Sed euismod semper hendrerit. Sed nec vulputate diam.

### Next Steps

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

* Maecenas in vulputate orci, eget posuere massa.
* Cras at lectus vel diam porta viverra.
* Ut vehicula congue aliquam. 